In [1]:
import pandas as pd
import numpy as np
from HFC125_emimTf2N_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.005
    init_kappa_1_2A = -0.01
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['emimTf2N'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['emimTf2N','emimTf2N'].fix(0)
    m.fs.properties.PR_kappa_A['emimTf2N', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'emimTf2N'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['emimTf2N', 'emimTf2N'].fix(0)
    m.fs.properties.PR_kappa_B['emimTf2N', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'emimTf2N'].fix(0)
    m.fs.properties.PR_kappa_C['emimTf2N', 'emimTf2N'].fix(0)
    m.fs.properties.PR_kappa_C['emimTf2N', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'emimTf2N'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['emimTf2N'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'emimTf2N'].fix(float(data['x_emimTf2N']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['emimTf2N'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['emimTf2N', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['emimTf2N', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'emimTf2N'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'emimTf2N'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['emimTf2N', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'emimTf2N']"]


In [3]:
data = pd.read_csv('R125_emimTf2N.csv')

print(data)

    temperature  pressure  x_R125  x_emimTf2N
0         283.1     10000   0.008       0.992
1         283.1    100000   0.085       0.915
2         283.1    199700   0.168       0.832
3         283.1    297700   0.250       0.750
4         283.1    399700   0.331       0.669
5         283.1    499900   0.413       0.587
6         283.1    599700   0.496       0.504
7         283.1    699900   0.583       0.417
8         283.1    799600   0.681       0.319
9         298.1     10200   0.006       0.994
10        298.1     99800   0.055       0.945
11        298.1    199700   0.109       0.891
12        298.1    299700   0.161       0.839
13        298.2    399600   0.213       0.787
14        298.2    499900   0.267       0.733
15        298.2    599700   0.317       0.683
16        298.1    800100   0.422       0.578
17        298.1    999800   0.521       0.479
18        323.1     10200   0.004       0.996
19        323.1     99800   0.030       0.970
20        323.1    200000   0.059 

In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_emimTf2N"] += data_["x_emimTf2N"] * noise_x
    data_.loc[data_['x_emimTf2N'] <= 0.0001, 'x_emimTf2N'] = 0.0001
    data_.loc[data_['x_emimTf2N'] >= 0.9999 , 'x_emimTf2N'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
-0.141472449262228
2022-03-07 16:23:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:23:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:23:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:23:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:23:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:23:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:23:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:23:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 

2022-03-07 16:25:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[emimTf2N,R125] = -0.011326550245685279
fs.properties.PR_kappa_A[R125,emimTf2N] = -0.011619312129973539
2
-0.07540920792656199
2022-03-07 16:25:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:25:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
20

2022-03-07 16:26:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:26:42 [INFO] idaes

2022-03-07 16:28:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:28:09 [INFO] idaes

2022-03-07 16:29:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:29:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:29:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:29:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:29:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:29:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:29:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:29:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:29:42 [INFO] idaes

...

2022-03-07 17:30:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:30:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:30:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:30:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:30:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:30:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:30:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasibl

2022-03-07 17:31:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:31:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:31:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:31:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:31:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:31:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:31:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:31:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:31:24 [INFO] idaes

2022-03-07 17:32:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:32:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:32:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:32:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:32:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:32:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:32:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:32:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:32:30 [INFO] idaes

2022-03-07 17:33:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:33:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:33:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:33:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:33:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:33:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:33:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:33:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:33:37 [INFO] idaes

2022-03-07 17:34:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:34:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:34:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:34:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:34:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:34:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 17:34:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasibl

In [5]:
print(results_params)

[[-0.05660206 -0.02338714]
 [-0.01132655 -0.01161931]
 [-0.03444855 -0.01737974]
 [-0.06962666 -0.027187  ]
 [-0.00699415 -0.01181649]
 [-0.06829806 -0.04728624]
 [-0.00715265 -0.01163585]
 [-0.00714164 -0.01164249]
 [-0.01664365 -0.01290325]
 [-0.03223013 -0.01680617]
 [-0.00562988 -0.01252416]
 [-0.00106392 -0.01339237]
 [-0.02277809 -0.01441372]
 [-0.01946753 -0.01359459]
 [-0.0143367  -0.01234337]
 [-0.00789467 -0.01102209]
 [-0.01457338 -0.01240061]
 [-0.00783287 -0.01111655]
 [-0.00777682 -0.01117079]
 [-0.03003132 -0.01624228]
 [-0.00784561 -0.01120115]
 [-0.00716017 -0.01163231]
 [-0.00167918 -0.01332827]
 [ 0.00224107 -0.01369052]
 [-0.00023847 -0.01347537]
 [-0.00745228 -0.01164105]
 [-0.00109596 -0.01338917]
 [-0.0046984  -0.01282797]
 [-0.03770637 -0.01823075]
 [ 0.01665766 -0.01422462]
 [-0.00663489 -0.011921  ]
 [-0.01285121 -0.01198514]
 [-0.06016223 -0.02440439]
 [ 0.00216053 -0.01368356]
 [-0.03182526 -0.016702  ]
 [-0.03428409 -0.01733706]
 [-0.03524386 -0.01758653]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_emimTf2N_10_2.csv')